In [15]:
#Dependencies
import pandas as pd
import gmaps
import requests
import os

#API key
from config import g_key

In [16]:
#Store csv 

city_file = os.path.join("Resources", "cities.csv")
city_data_df = pd.read_csv(city_file)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rawson,AR,2022-01-24 04:00:56,-43.3002,-65.1023,68.40,39,39,18.54
1,1,Isla Vista,US,2022-01-24 04:00:31,34.4133,-119.8610,61.68,65,0,4.61
2,2,Bisho,ZA,2022-01-24 04:00:57,-32.8472,27.4422,69.48,43,0,5.28
3,3,Yerofey Pavlovich,RU,2022-01-24 04:00:57,53.9667,121.9667,6.49,67,6,2.84
4,4,Petropavl,KZ,2022-01-24 04:00:58,54.8753,69.1628,-14.82,76,19,2.24


In [17]:
#Check those types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [18]:
#Config gmaps to use API key
#Process to create a heat map is as follows:
# 1. Assign the locations to an array of latitude and longitude pairs.
#locations = [latitude, longitude]
# 2. Assign the weights variable to some values.
#temperatures = # an array of length equal to the locations array length
# 3. Assign the figure variable to the gmaps.figure() attribute.
#fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
#heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)
# 5. Add the heatmap layer.
#fig.add_layer(heatmap_layer)
# 6. Call the figure to plot the data.
#fig


In [19]:
locations = city_data_df[["Lat", "Lng"]]

max_temp = city_data_df["Max Temp"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

fig.add_layer(heat_layer)

#Call the figure
fig

InvalidWeightException: -14.82 is not a valid weight. Weights must be non-negative.

In [20]:
#This throws an error because max_temp values cannot be negative, so let's make a for loop to get rid of the negative values
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    #using max() to get largest value b/w temp and 0. If temp < 0 , then 0 will be added to the list in its place,
    #otherwise temp is added to the list
    temps.append(max(temp,0))

In [21]:
#Trying again, replacing max_temp with temps

locations = city_data_df[["Lat", "Lng"]]

max_temp = city_data_df["Max Temp"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=temps)

fig.add_layer(heat_layer)

#Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
#Instead of that for loop, let's use the more efficent "list comprehension" method
#Also we will center the map in the fig variable
#Also modify the g.maps_layer attribute

locations = city_data_df[["Lat", "Lng"]]

max_temp = city_data_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

#Call the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
#Let's get humid
locations = city_data_df[["Lat","Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 89


In [28]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rawson,AR,2022-01-24 04:00:56,-43.3002,-65.1023,68.40,39,39,18.54
1,1,Isla Vista,US,2022-01-24 04:00:31,34.4133,-119.8610,61.68,65,0,4.61
2,2,Bisho,ZA,2022-01-24 04:00:57,-32.8472,27.4422,69.48,43,0,5.28
6,6,Nacimiento,CL,2022-01-24 04:00:58,-37.5000,-72.6667,56.23,78,0,6.17
7,7,Chitradurga,IN,2022-01-24 04:00:59,14.2333,76.4000,63.39,84,84,3.47
8,8,Pangoa,PE,2022-01-24 04:00:59,-12.1167,-73.0000,67.44,99,46,1.14
10,10,Puerto Lempira,HN,2022-01-24 04:01:00,15.2667,-83.7667,74.41,88,0,8.28
11,11,Adrar,MR,2022-01-24 04:01:01,20.5022,-10.0711,61.90,19,83,10.56
12,12,Galle,LK,2022-01-24 04:01:02,6.0367,80.2170,79.23,73,100,4.94
13,13,Manakara,MG,2022-01-24 04:01:02,-22.1333,48.0167,77.25,89,78,14.12


In [29]:
preferred_cities_df.count()

City_ID       66
City          66
Country       66
Date          66
Lat           66
Lng           66
Max Temp      66
Humidity      66
Cloudiness    66
Wind Speed    66
dtype: int64

In [30]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rawson,AR,68.40,-43.3002,-65.1023,
1,Isla Vista,US,61.68,34.4133,-119.8610,
2,Bisho,ZA,69.48,-32.8472,27.4422,
6,Nacimiento,CL,56.23,-37.5000,-72.6667,
7,Chitradurga,IN,63.39,14.2333,76.4000,
8,Pangoa,PE,67.44,-12.1167,-73.0000,
10,Puerto Lempira,HN,74.41,15.2667,-83.7667,
11,Adrar,MR,61.90,20.5022,-10.0711,
12,Galle,LK,79.23,6.0367,80.2170,
13,Manakara,MG,77.25,-22.1333,48.0167,


In [33]:
#Let's make an API call to the places API for google maps. We will need to set certain parameters
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [40]:
#Let's bypass the error code
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rawson,AR,68.40,-43.3002,-65.1023,Hosteria Sampedro
1,Isla Vista,US,61.68,34.4133,-119.8610,Best Western Plus South Coast Inn
2,Bisho,ZA,69.48,-32.8472,27.4422,Bisho Park bnb
6,Nacimiento,CL,56.23,-37.5000,-72.6667,Residencial San Marcos
7,Chitradurga,IN,63.39,14.2333,76.4000,V. G. S. Comforts
8,Pangoa,PE,67.44,-12.1167,-73.0000,
10,Puerto Lempira,HN,74.41,15.2667,-83.7667,Puerto Lempira gracias a Dios
11,Adrar,MR,61.90,20.5022,-10.0711,
12,Galle,LK,79.23,6.0367,80.2170,Amangalla
13,Manakara,MG,77.25,-22.1333,48.0167,La Residence Manakara


In [51]:
# Add a heatmap of temperature for the vacation spots to the new hotel df.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
#Once the markers are added, we can add a pop-up for each marker to display the corresponding hotel information
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{column1}</dd>
#<dt>Another name</dt><dd>{column2}</dd>
#</dl>
#"""

#Let's specify it to the hotels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [50]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=500, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))